In [24]:
import shutil
import os
import tempfile
from langchain.document_loaders import TextLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.vectorstores import Chroma
from langchain_community.embeddings import HuggingFaceEmbeddings
import os
import google.generativeai as genai

# Replace 'YOUR_API_KEY' with your actual API key
API_KEY = '
# Configure the API key
genai.configure(api_key=API_KEY)

# Initialize the model
model = genai.GenerativeModel('gemini-1.5-flash')

# Clear the existing Chroma database
if os.path.exists("chroma_db"):
    shutil.rmtree("chroma_db", ignore_errors=True)

# Load the text file
loader = TextLoader('text.txt')  # Replace with your file path
documents = loader.load()

# Split the text into smaller chunks
text_splitter = RecursiveCharacterTextSplitter(chunk_size=100, chunk_overlap=10)
docs = text_splitter.split_documents(documents)

# Use embeddings designed for semantic similarity
embeddings = HuggingFaceEmbeddings(model_name="sentence-transformers/multi-qa-mpnet-base-dot-v1")

# Use a temporary directory for Chroma database
persist_directory = tempfile.mkdtemp()
vectorstore = Chroma.from_documents(
    docs,
    embeddings,
    persist_directory=persist_directory
)

# Create a retriever from the vector store
retriever = vectorstore.as_retriever(search_kwargs={"k": 1})

# Example query
query = input("Query:")  # Replace with your question

# Retrieve relevant documents
relevant_docs = retriever.get_relevant_documents(query)

# Print only the most relevant documents
print("Relevant Information:")
for doc in relevant_docs:
    print(doc.page_content +"\n")

# Start a chat session
chat = model.start_chat()

rag_sysin = "you simply answer questions by using knowledge from a data store. this is your data:"

# Prompt the user for a question
user_question = rag_sysin + doc.page_content + "and here's your query" + query 

# Send the user's question to the model
response = chat.send_message(user_question)

# Print the model's response
print("Response:", response.text)


Relevant Information:
Sheldon's Formula:
Slavery = profit
devised by sheldon periperi

Response: Sheldon Periperi



In [ ]:
import shutil
import os
import tempfile
from PyPDF2 import PdfReader  # Library for reading PDFs
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.vectorstores import Chroma
from langchain_community.embeddings import HuggingFaceEmbeddings
import google.generativeai as genai

# Replace 'YOUR_API_KEY' with your actual API key
API_KEY = '
# Configure the API key
genai.configure(api_key=API_KEY)

# Initialize the model
model = genai.GenerativeModel('gemini-1.5-flash')

# Function to extract text from a PDF file
def extract_text_from_pdf(pdf_path):
    try:
        reader = PdfReader(pdf_path)
        text = ""
        for page in reader.pages:
            text += page.extract_text()
        return text
    except Exception as e:
        print(f"Error extracting text from PDF: {e}")
        exit(1)

# Load the PDF file
pdf_path = 'formulas1.pdf'  # Replace with your PDF file path
if not os.path.exists(pdf_path):
    print("Error: PDF file not found!")
    exit(1)

# Extract text from the PDF
pdf_text = extract_text_from_pdf(pdf_path)

# Split the text into smaller chunks
text_splitter = RecursiveCharacterTextSplitter(chunk_size=200, chunk_overlap=10)
docs = text_splitter.create_documents([pdf_text])

# Use embeddings designed for semantic similarity
embeddings = HuggingFaceEmbeddings(model_name="sentence-transformers/multi-qa-mpnet-base-dot-v1")

# Use a temporary directory for Chroma database
persist_directory = tempfile.mkdtemp()
vectorstore = Chroma.from_documents(
    docs,
    embeddings,
    persist_directory=persist_directory
)

# Create a retriever from the vector store
retriever = vectorstore.as_retriever(search_kwargs={"k": 4})

In [ ]:
# Example query
query = input("Query: ")  # User-provided question

# Retrieve relevant documents
relevant_docs = retriever.get_relevant_documents(query)
combined_content = " ".join([doc.page_content for doc in relevant_docs])

# Print relevant information
print("Relevant Information:")
print(combined_content + "\n")

# Start a chat session
chat = model.start_chat()

# Create a structured prompt
user_question = f"""
You are an AI system answering queries based on a data store. 
Here is the data you should use:
{combined_content}

Here is the user's query:
{query}

Please provide a detailed and accurate answer.
"""

try:
    # Send the user's question to the model
    response = chat.send_message(user_question)
    print("Response:", response.text)
except Exception as e:
    print(f"Error occurred: {e}")

# Clean up temporary directories
shutil.rmtree(persist_directory, ignore_errors=True)